Installing Necessary Packages

In [2]:
!pip install chainlit ctransformers torch sentence_transformers chromadb langchain pypdf PyPDF2 langchain-community gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metad

Import Packages, downloading HuggingFaceBgeEmbeddings model for the first time

In [3]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
import gradio as gr

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Loading PDF and Chunking. Extracted only 2 chapters as mentioned in the document for faster indexing as a prerequisite

In [5]:
loader = PyPDFLoader("/BioText.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

Creating Vectors for the loaded PDF and storing embeddings in persisted directory

NOTE: Please specify your persist directory path while running this cell

In [6]:
vector_store = Chroma.from_documents(texts, embeddings, collection_metadata={"hnsw:space": "cosine"}, persist_directory="/content/stores/biology_cosine")

print("Vector Store Created.......")

Vector Store Created.......


- The script begins by setting up the local LLM model and its configuration parameters.
- A prompt template is defined, guiding how the LLM should respond to user queries.
- A pre-trained embedding model is used to manage the text-to-vector transformation.
- A vector store containing biology content is loaded for retrieving relevant context during queries.
- **local_llm**: Specifies the local model file used for the LLM.
- **config**: A dictionary containing settings for the LLM, such as token generation limits, temperature, and threading options.
- **llm**: Initializes the LLM model with the specified configuration using the CTransformers library.
- **prompt_template**: Defines the structure and rules for generating responses from the LLM, including how to use the context and question.
- **model_name, model_kwargs, encode_kwargs**: Parameters for the HuggingFaceBge Embeddings model that converts text into vector embeddings.
- **embeddings**: The embedding function initialized with the specified model and settings.
- **load_vector_store**: Loads the Chroma vector store with precomputed embeddings for biology-related content.
- **retriever**: A retriever object for fetching relevant documents from the vector store based on the user's query.



**NOTE**: As the model is heavy it is mounted from Google Drive. Please download model "zephyr-7b-alpha.Q5_K_S.gguf "on your local using below link and give your model path before running this notebook

**Model Link:**
https://huggingface.co/TheBloke/zephyr-7B-alpha-GGUF/tree/main

Please specify *your persisted directory* path to load generated embeddings

In [7]:
local_llm = "/content/drive/MyDrive/Colab Notebooks/zephyr-7b-alpha.Q5_K_S.gguf"

config = {
'max_new_tokens': 1024,
'repetition_penalty': 1.1,
'temperature': 0.1,
'top_k': 50,
'top_p': 0.9,
'stream': True,
'threads': int(os.cpu_count() / 2)
}

llm = CTransformers(
    model=local_llm,
    model_type="mistral",
    lib="avx2", #for CPU use
    **config
)

print("LLM Initialized...")


prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
load_vector_store = Chroma(persist_directory="/content/stores/biology_cosine", embedding_function=embeddings)
retriever = load_vector_store.as_retriever(search_kwargs={"k":1})

chain_type_kwargs = {"prompt": prompt}

# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     return_source_documents = True,
#     chain_type_kwargs= chain_type_kwargs,
#     verbose=True
# )

# response = qa(query)

# print(response)

LLM Initialized...
######################################################################


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [8]:
sample_prompts = ["what is Chemical Bond?", "Explain Evolution of life?"]

In [9]:
# def get_response(input):
#   print("entered response")
#   query = input
#   chain_type_kwargs = {"prompt": prompt}
#   qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True, chain_type_kwargs=chain_type_kwargs, verbose=True)
#   print("docs retrieved")
#   response = qa.invoke(query)
#   print("Response generated")
#   print(response)
#   return response

# get_response("Explain about Chemical bonds")

- **get_response**: A function that handles the user query, retrieves relevant documents, and generates a response using the LLM.
- **iface**: A Gradio interface that provides a web-based UI for interacting with the model. It accepts user input, runs the get_response function, and displays the output.

Example Usage:
--------------
- The script includes sample prompts such as "What is Carbohydrate?" and "What is Chemical Bond?".
- Users can interact with the bot through a simple web interface, inputting questions related to biology and receiving contextual answers.

Deployment:
-----------
- The script can be run locally, with Gradio launching a web interface for user interaction.
- Debugging options are available through Gradio's `launch()` function, where errors can be inspected by setting `debug=True`.


In [ ]:
def get_response(input):
  print("entered response")
  query = input
  chain_type_kwargs = {"prompt": prompt}
  qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True, chain_type_kwargs=chain_type_kwargs, verbose=True)
  print("docs retrieved")
  response = qa(query)
  print("Response generated")
  return response

input = gr.Text(
                label="Prompt",
                show_label=False,
                max_lines=1,
                placeholder="Enter your prompt",
                container=False,
            )

iface = gr.Interface(fn=get_response,
             inputs=input,
             outputs="text",
             title="My Biology Bot",
             description="This is a RAG implementation based on Zephyr 7B Beta LLM.",
             examples=sample_prompts,
             allow_flagging=False
             )

iface.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:390: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3f73c5e8b5b6276036.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


entered response
docs retrieved


> Entering new RetrievalQA chain...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.
Response generated
entered response
docs retrieved


> Entering new RetrievalQA chain...

> Finished chain.
Response generated


**Assumptions:**

1. To extract only 2 chapters from the PDF I have used python code. I assumed that as prerequisite and not included in the notebook
2. Assumed free to use any LLM and Embeddings models.

**Challenges:**
1. Getting access to open source LLM models like Llama3, Llama2 etc. Though it is a Opensource Meta team's approval is required.
2. Memory constraint to load models on premise.
3. Can use API Interface of Hugging Face but faced No access issue though token is generated. Sometimes, faced too many requests issue.
4. Each time only a document is retrieved for a query due to memory and speed constraint(k=1).
5. Response Time is more on CPU.
6. Embedding Creations has taken more time on CPU
7. When a framework version changes, the respective imports has changed especially with langchain framework.
8. Could have implemented using Class. I didn't implement as I had very less time to do so. Class could have been compact and reduced repetition of code especially defining embedding model.

**Produce alternative way to do the RAG without using any library like Langchain, LLamaIndex or Haystack**

1. Loading and Reading PDF
2. Chunking has to be done with customized python code
3. Embeddings can be created using pre-trained sentence transformer model
4. Embeddings to be stored in In-Memory Vector Store
5. For Retrieval, Query to be vectorized and perform  similarity search on vector db.
6. Pass Retrieved docs and query to LLM Model to generate response

